In [1]:
import pandas as pd
import math
import json

In [2]:

with open('network_data/grid_island.json') as json_data:
    grid_islands = json.load(json_data)

#grid_islands = pd.DataFrame.from_dict(grid_islands, orient='columns', dtype=None)


grid = pd.DataFrame.from_dict(grid_islands['9'])
grid = grid.rename(columns = {0:'NodeID'}) 


grid




,NodeID
0,52034625
1,42882929


In [3]:
# Transformer input

transformer = pd.read_csv('network_data/transformer_combined.csv')

transformer = transformer[["SectionID" , "KVA", "KVLLprim", "KVLLsec"]].copy()

transformer[["SectionID"]] = transformer[["SectionID"]].astype(int)
transformer[["SectionID"]] = transformer[["SectionID"]].astype(str) # to get rid of the .0 put keep it as string

transformer

,SectionID,KVA,KVLLprim,KVLLsec,FromNodeID,ToNodeID
0,42547360,400,10.6,0.42,NaN,NaN
1,42571987,630,10.6,0.42,NaN,NaN
2,42590298,400,10.6,0.42,NaN,NaN
3,42552624,100,10.6,0.42,NaN,NaN
4,42574167,630,21.0,0.42,NaN,NaN
5,214551743,630,21.0,0.42,NaN,NaN
6,42595956,10000,21.0,10.60,NaN,NaN
7,42613835,630,21.0,0.42,NaN,NaN
8,42571862,630,10.6,0.42,NaN,NaN
9,42574494,630,21.0,0.42,NaN,NaN


In [4]:
### source equivalent

source_equivalent = pd.read_csv('network_data/source_equivalent.csv')

source_equivalent = source_equivalent[source_equivalent['LoadModelName'] == 'MOYENNE']
source_equivalent = source_equivalent[['NodeID', 'Voltage']].reset_index()

source_equivalent


,index,NodeID,Voltage
0,2,193621144,21.00
1,5,42890827,0.42
2,8,52034725,21.00
3,11,42975166,0.42
4,14,42889208,0.42
5,17,42948116,0.42
6,20,88867752,0.42
7,23,42875734,0.42
8,26,175587451,0.42
9,29,148912587,0.42


In [5]:
grid['NodeID'] = grid['NodeID'].astype(str)
source_equivalent['NodeID'] = source_equivalent['NodeID'].astype(str)

source_grid = source_equivalent[source_equivalent['NodeID'].isin(grid['NodeID'])]

#source_grid.to_csv(r'source_mid_grid.csv', header=True, index=True, mode='a')

In [6]:
### Writing an algo to check the connections 

### Section ###

section_orig = pd.read_csv('../preprocesses/section_data/result_section_better.csv')
#section['SectionID'] ==  
section = section_orig[["SectionID", "FromNodeID", "ToNodeID"]].copy()

#section = section.rename(index=str, columns={"": "", "": ""})

section['SectionID'] = section['SectionID'].astype(str)
section['FromNodeID'] = section['FromNodeID'].astype(str)
section['ToNodeID'] = section['ToNodeID'].astype(str)

section = section[~section['SectionID'].isin(transformer['SectionID'])]
section_2 = section[section['FromNodeID'].isin(grid['NodeID'])]
section_3 = section[section['ToNodeID'].isin(grid['NodeID'])]


frames = [section_2,section_3] 
section_island = pd.concat(frames)

section_island=section_island.drop_duplicates()

section_is = section_island.rename(columns = {'ToNodeID':'NodeID'}) 
section_is = section_is.merge(source_equivalent, on='NodeID')
section_is = section_is.rename(columns = {'NodeID':'ToNodeID'})


section_is_2 = section_island.rename(columns = {'FromNodeID':'NodeID'})
section_is_2 = section_is_2.merge(source_equivalent, on='NodeID')
section_is_2 = section_is_2.rename(columns = {'NodeID':'FromNodeID'})

frames = [section_is,section_is_2] 
section_is = pd.concat(frames)

section_is = section_is.drop_duplicates().reset_index()
section_is = section_is[['SectionID', 'FromNodeID', 'ToNodeID', 'Voltage']]

section_is

,SectionID,FromNodeID,ToNodeID,Voltage


In [7]:
section_is[section_is['Voltage']==0.42]

,SectionID,FromNodeID,ToNodeID,Voltage


In [8]:
section_is[section_is['Voltage']==10.60]

,SectionID,FromNodeID,ToNodeID,Voltage


In [9]:
section_is[section_is['Voltage']==21.00]

,SectionID,FromNodeID,ToNodeID,Voltage
